In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Preprocess data
(only needs to be done once)

In [5]:
from pre import preprocess
# stores preprocessed data file
preprocessed_filename = './data/preprocessed.csv'
preprocess(i='./data/training.1600000.processed.noemoticon.csv', o=preprocessed_filename, slice=1000)

Loading ./data/training.1600000.processed.noemoticon.csv


  0%|          | 0/1000 [00:00<?, ?it/s]

Stripping down
Preprocessing


100%|██████████| 1000/1000 [00:06<00:00, 151.32it/s]


Done. Writing to ./data/preprocessed.csv
Success!


### Load preprocessed data

In [6]:
from parse import load_data

data = load_data(preprocessed_filename)
# TODO: train/dev/test split

### Embedding

In [7]:
from keras.preprocessing.text import Tokenizer
from emb import get_keras_embeddings_layer

MAX_WORDS = 5000
glove_filename = './data/glove.6B.50d.txt'
tokenizer = Tokenizer(lower=True, split=' ', oov_token="UNK")
emb_layer = get_keras_embeddings_layer(glove_filename, preprocessed_filename, tokenizer)

got tweets
Glove data loaded
Words not found 521


### Model

In [28]:
%load_ext autoreload
%autoreload 2
from TextClassifierModel import build_model_keras, KerasTextClassifier
import pickle


# TODO: check correct arg-assignment for tokenizer
model = build_model_keras(tokenizer, emb_layer)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
building model
init
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  141600    
_________________________________________________________________
flatten_6 (Flatten)          (None, 5000)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 5000)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 64)                320064    
_________________________________________________________________
dense_13 (Dense)             (None, 100)               6500      
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 202       
Total params

### Save untrained model | load it

### Train

In [29]:
%autoreload 2
def mapping(n):
            if n == 0:
                return 0
            if n == 4:
                return 1


data_0 = data[data['target']==0]
data_4 = data[data['target']==4]

count_0, count_4 = data.target.value_counts()
print(count_0, count_4)



split = len(data['tweet'])//2

training_data = np.array(data['tweet'][:split])
training_target = np.array(data['target'][:split])
test_data = np.array(data['tweet'][split:])
test_target = np.array(data['target'][split:])

for i in range(len(training_target)):
    training_target[i] = mapping(training_target[i])
for i in range(len(test_target)):
    test_target[i] = mapping(test_target[i])
training_target = tf.keras.utils.to_categorical(training_target, 2)
test_target = tf.keras.utils.to_categorical(test_target, 2)

#print(training_target, test_target)
#doc = np.array(data['tweet'][:1])



507 493


In [30]:
%load_ext autoreload
%autoreload 2

history = model.fit(training_data, training_target, epochs=4, batch_size=30)
# TODO: save trained model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Epoch 1/4
15/15 [==============================] - 1s 30ms/step - loss: 1.7334 - accuracy: 0.4797 - val_loss: 1.0560 - val_accuracy: 0.4800
Epoch 2/4
15/15 [==============================] - 0s 12ms/step - loss: 0.9093 - accuracy: 0.6152 - val_loss: 0.8008 - val_accuracy: 0.5000
Epoch 3/4
15/15 [==============================] - 0s 13ms/step - loss: 0.7177 - accuracy: 0.7080 - val_loss: 0.7703 - val_accuracy: 0.5000
Epoch 4/4
15/15 [==============================] - 0s 12ms/step - loss: 0.6826 - accuracy: 0.6418 - val_loss: 0.7555 - val_accuracy: 0.5200


### Evaluate dev

### Save model

### Evaluate production (forbidden during tuning)

### Explain prediction

In [17]:
import eli5
from eli5.lime import TextExplainer

In [19]:



te = TextExplainer(random_state=42)
te.fit("I love candy. I like to be positive, be happy! What a lovely day", model.predict_proba)
te.show_prediction()